In [59]:
import os
import openai
#import chromadb
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate,MessagesPlaceholder

In [60]:
os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2023-03-15-preview" #"2022-12-01" 
os.environ["OPENAI_API_BASE"] = openai.api_base =  "https://idocopenaigpt.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = openai.api_key = "95776649ac7a4b048c834003fd315264"#os.getenv("AZUREOPENAI_API_KEY")

embeddings = AzureOpenAIEmbeddings(deployment="idocembedd")

c:\Users\Sukrit\Documents\Projects\Lancgchain_demo\venv\lib\site-packages\langchain_community\embeddings\azure_openai.py:112: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://idocopenaigpt.openai.azure.com/ to https://idocopenaigpt.openai.azure.com//openai.
  warnings.warn(
c:\Users\Sukrit\Documents\Projects\Lancgchain_demo\venv\lib\site-packages\langchain_community\embeddings\azure_openai.py:119: UserWarning: As of openai>=1.0.0, if `deployment` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\Sukrit\Documents\Projects\Lancgchain_demo\venv\lib\site-packages\langchain_community\embeddings\azure_openai.py:127: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified

In [61]:
from langchain.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain import PromptTemplate, LLMChain

In [62]:
model = AzureChatOpenAI(
    openai_api_base="https://idocopenaigpt.openai.azure.com/",
    openai_api_version="2023-03-15-preview",
    deployment_name='chatllm16k',
    openai_api_key="95776649ac7a4b048c834003fd315264",
    openai_api_type="azure",
    temperature=0
)

c:\Users\Sukrit\Documents\Projects\Lancgchain_demo\venv\lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://idocopenaigpt.openai.azure.com/ to https://idocopenaigpt.openai.azure.com/openai.
  warnings.warn(
c:\Users\Sukrit\Documents\Projects\Lancgchain_demo\venv\lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\Sukrit\Documents\Projects\Lancgchain_demo\venv\lib\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) 

In [63]:
from langchain.memory import ConversationBufferMemory

In [64]:
memory = ConversationBufferMemory()

In [65]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI

In [66]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

In [130]:
Template = """You are an AI assistant for IT support team. 
Your primary job is to infer title, description and category from user input and create the ticket.
Do the following steps:

1. Don't send any greet message if user is talking about issue.
2. Ask for the issue details.
3. Infer title,description and category.
4. title,description and category should be clear to you. In case if you won't able to infer title,description and category then you should ask for more details regarding issue.
5. Get the confirmation from user by saying "Thanks for the details. Below is your details." Mention the issue details in the structured format and ask the user "Please confirm if I can go ahead and create a ticket with the mentioned details.
6.Once user confirms then return the message "Thanks for confirming! I'll cretae a ticket with following issue details." and issue details in JSON format. 
7. Your output should be specific to user input. Think wisely before providing any output.
8. Don't create ticket for the same issue again and again.




User input: {description}


LLM Output: Thanks for the details, I'll create a ticket for you !!


"""


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=Template
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{description}"
        ),  # Where the human input will injected
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [131]:
chat_llm_chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [132]:
chat_llm_chain.predict(description="Hi! I'm facing issues")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI assistant for IT support team. 
Your primary job is to infer title, description and category from user input and create the ticket.
Do the following steps:

1. Don't send any greet message if user is talking about issue.
2. Ask for the issue details.
3. Infer title,description and category.
4. title,description and category should be clear to you. In case if you won't able to infer title,description and category then you should ask for more details regarding issue.
5. Get the confirmation from user by saying "Thanks for the details. Below is your details." Mention the issue details in the structured format and ask the user "Please confirm if I can go ahead and create a ticket with the mentioned details.
6.Once user confirms then return the message "Thanks for confirming! I'll cretae a ticket with following issue details." and issue details in JSON format. 
7. Your output should be specific to user input. 

"Sure, I'm here to help. Can you please provide more details about the issues you are facing?"

In [133]:
response = chat_llm_chain.predict(description="I'm not able to connect my laptop to the internet.I'm not able to install any Application in my laptop. Also my PC is working very slow as well and lagging a lot.")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI assistant for IT support team. 
Your primary job is to infer title, description and category from user input and create the ticket.
Do the following steps:

1. Don't send any greet message if user is talking about issue.
2. Ask for the issue details.
3. Infer title,description and category.
4. title,description and category should be clear to you. In case if you won't able to infer title,description and category then you should ask for more details regarding issue.
5. Get the confirmation from user by saying "Thanks for the details. Below is your details." Mention the issue details in the structured format and ask the user "Please confirm if I can go ahead and create a ticket with the mentioned details.
6.Once user confirms then return the message "Thanks for confirming! I'll cretae a ticket with following issue details." and issue details in JSON format. 
7. Your output should be specific to user input. 


> Finished chain.
Thanks for the details. Based on the information provided, I have inferred the following details:

Title: Issues with laptop connectivity, application installation, and performance
Description: The user is unable to connect their laptop to the internet, install any applications, and is experiencing slow performance and lagging.

Category: Network, Software, Performance

Please confirm if I can go ahead and create a ticket with the mentioned details.


In [134]:
response = chat_llm_chain.predict(description="yes")
print(response)





> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI assistant for IT support team. 
Your primary job is to infer title, description and category from user input and create the ticket.
Do the following steps:

1. Don't send any greet message if user is talking about issue.
2. Ask for the issue details.
3. Infer title,description and category.
4. title,description and category should be clear to you. In case if you won't able to infer title,description and category then you should ask for more details regarding issue.
5. Get the confirmation from user by saying "Thanks for the details. Below is your details." Mention the issue details in the structured format and ask the user "Please confirm if I can go ahead and create a ticket with the mentioned details.
6.Once user confirms then return the message "Thanks for confirming! I'll cretae a ticket with following issue details." and issue details in JSON format. 
7. Your output should be specific to user input. 

In [135]:
response = chat_llm_chain.predict(description="No,Thanks!")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI assistant for IT support team. 
Your primary job is to infer title, description and category from user input and create the ticket.
Do the following steps:

1. Don't send any greet message if user is talking about issue.
2. Ask for the issue details.
3. Infer title,description and category.
4. title,description and category should be clear to you. In case if you won't able to infer title,description and category then you should ask for more details regarding issue.
5. Get the confirmation from user by saying "Thanks for the details. Below is your details." Mention the issue details in the structured format and ask the user "Please confirm if I can go ahead and create a ticket with the mentioned details.
6.Once user confirms then return the message "Thanks for confirming! I'll cretae a ticket with following issue details." and issue details in JSON format. 
7. Your output should be specific to user input. 


> Finished chain.
You're welcome! If you have any more issues in the future, feel free to reach out. Have a great day!


In [136]:
def create_ticket(title, description, category):
    # Implement logic to interact with your IT help desk API
    # Send a request to create a ticket with title, description, and category
    # Extract the ticket details (ticket number, link) from the API response
    ticket_number = "12345"  # Replace with actual ticket number
    ticket_link = "https://helpdesk.example.com/tickets/12345"  # Replace with actual ticket link
    return ticket_number, ticket_link

In [121]:
import json
import re
#response = chat_llm_chain.predict(description="I'm not able to connect my laptop to the internet.I'm not able to install any Application in my laptop. Also my PC is working very slow as well and lagging a lot.")
#print(response)

json_match = re.search(r'{.*}', response)
json_part = json_match.group(0) if json_match else None


response_dict = json.loads(json_part) if json_part else {}

#print(response_dict)



{}


In [122]:
title = response_dict.get("title", "Default Title")
description = response_dict.get("Description", "Default Description")
category = response_dict.get("Category", "Default Category")

# Create a ticket in the IT help desk
ticket_number, ticket_link = create_ticket(title, description, category)

# Provide the user with ticket details
print(f"Your ticket has been created!\nTicket Number: {ticket_number}\nTicket Link: {ticket_link}")

Your ticket has been created!
Ticket Number: 12345
Ticket Link: https://helpdesk.example.com/tickets/12345


In [94]:
print("LLM Response:", response)

print("LLM Response:", response)
print("Type of response:", type(response))

# Check if the response is a string
if isinstance(response, str):
    print("Response is a string. Investigate further to understand its structure.")
else:
    # Extract values based on the correct keys if response is a dictionary
    title = response.get("Title", "Default Title")
    description = response.get("Description", "Default Description")
    category = response.get("Category", "Default Category")

    # Create a ticket in the IT help desk
    ticket_number, ticket_link = create_ticket(title, description, category)

    # Provide the user with ticket details
    print(f"Your ticket has been created!\nTicket Number: {ticket_number}\nTicket Link: {ticket_link}")

LLM Response: Thanks for confirming! I'll create a ticket with the following issue details:

{
  "title": "Issues with laptop connectivity, application installation, and performance",
  "description": "The user is unable to connect their laptop to the internet, install any applications, and is experiencing slow performance and lagging.",
  "category": "Network/Internet, Software Installation, Performance"
}

Please let me know if there's anything else I can assist you with.
LLM Response: Thanks for confirming! I'll create a ticket with the following issue details:

{
  "title": "Issues with laptop connectivity, application installation, and performance",
  "description": "The user is unable to connect their laptop to the internet, install any applications, and is experiencing slow performance and lagging.",
  "category": "Network/Internet, Software Installation, Performance"
}

Please let me know if there's anything else I can assist you with.
Type of response: <class 'str'>
Response i

In [95]:
title = response["title"]  # Extract title from LLM response
description = response["description"]  # Extract description from LLM response
category = response["category"]  # Extract category from LLM response

# Create a ticket in the IT help desk
ticket_number, ticket_link = create_ticket(title, description, category)


TypeError: string indices must be integers